Dynamic Pricing for Urban Parking Lots
INTRODUCTION:
This project builds a real-time dynamic pricing system for urban parking lots to improve space utilization and revenue. Using features like occupancy, queue length, traffic, and special days, we develop intelligent pricing models.

Tech Stack: Python, Pandas, NumPy, Pathway (for real-time data), Bokeh (for visualization)

We implement:

Model 1: Baseline linear pricing based on occupancy

Model 2: Demand-based pricing using multiple features

Model 3: Competitive pricing using geolocation


MODEL 1

In [ ]:
#Import required libraries
import pandas as pd
import numpy as np

#Step 1: Load the dataset
df = pd.read_csv("dataset.csv")

#Step 2: Combine date and time into a proper Timestamp
# Fix format: Date is dd-mm-yyyy and time is HH:MM:SS
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S',
    errors='coerce'  # if any invalid date, it becomes NaT
)

# Remove rows with invalid timestamps
df = df.dropna(subset=['Timestamp'])

#Step 3: Sort data by Parking Lot and Timestamp
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp'])

#Step 4: Set baseline pricing parameters
base_price = 10.0     # starting price
alpha = 2.0           # sensitivity factor
min_price = 5.0       # minimum allowed price
max_price = 20.0      # maximum allowed price

#Create a new column for baseline prices
df['BaselinePrice'] = np.nan

#Step 5: Loop through each parking lot and compute price over time
all_lots = []

for lot_id, lot_df in df.groupby('SystemCodeNumber'):
    lot_df = lot_df.sort_values('Timestamp').copy()
    
    prices = [base_price]  # initialize with base price

    for i in range(1, len(lot_df)):
        prev_price = prices[-1]
        occupancy = lot_df.iloc[i - 1]['Occupancy']
        capacity = lot_df.iloc[i - 1]['Capacity']

        # Simple linear update rule
        next_price = prev_price + alpha * (occupancy / capacity)

        # Clamp price within bounds
        next_price = max(min_price, min(max_price, next_price))
        prices.append(next_price)

    # Store computed prices in the dataframe
    lot_df.loc[:, 'BaselinePrice'] = prices
    all_lots.append(lot_df)

#Step 6: Combine all results
df_model1 = pd.concat(all_lots)

#Step 7: Preview the final output
df_model1[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'BaselinePrice']].head(10)


,SystemCodeNumber,Timestamp,Occupancy,Capacity,BaselinePrice
0,BHMBCCMKT01,2016-10-04 07:59:00,61,577,10.000000
1,BHMBCCMKT01,2016-10-04 08:25:00,64,577,10.211438
2,BHMBCCMKT01,2016-10-04 08:59:00,80,577,10.433276
3,BHMBCCMKT01,2016-10-04 09:32:00,107,577,10.710572
4,BHMBCCMKT01,2016-10-04 09:59:00,150,577,11.081456
5,BHMBCCMKT01,2016-10-04 10:26:00,177,577,11.601386
6,BHMBCCMKT01,2016-10-04 10:59:00,219,577,12.214905
7,BHMBCCMKT01,2016-10-04 11:25:00,247,577,12.974003
8,BHMBCCMKT01,2016-10-04 11:59:00,259,577,13.830156
9,BHMBCCMKT01,2016-10-04 12:29:00,266,577,14.727903


MODEL 2


In [ ]:
# Required imports
import pandas as pd
import numpy as np

# Use your previously cleaned df_model1 as base
df_model2 = df_model1.copy()

#Vehicle type weights
vehicle_type_weights = {
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5
}
df_model2['VehicleWeight'] = df_model2['VehicleType'].map(vehicle_type_weights)

#Traffic level encoding
traffic_level_weights = {
    'low': 1,
    'medium': 2,
    'high': 3
}
df_model2['TrafficLevel'] = df_model2['TrafficConditionNearby'].map(traffic_level_weights)

#Parameters for demand function
alpha = 1.5     # Occupancy weight
beta = 0.8      # Queue length
gamma = 1.0     # Traffic penalty
delta = 2.0     # Special day boost
epsilon = 1.0   # Vehicle type importance

#Step 1: Compute raw demand
df_model2['RawDemand'] = (
    alpha * (df_model2['Occupancy'] / df_model2['Capacity']) +
    beta * df_model2['QueueLength'] -
    gamma * df_model2['TrafficLevel'] +
    delta * df_model2['IsSpecialDay'] +
    epsilon * df_model2['VehicleWeight']
)

#Step 2: Normalize demand between 0 and 1
min_d = df_model2['RawDemand'].min()
max_d = df_model2['RawDemand'].max()
df_model2['NormalizedDemand'] = (df_model2['RawDemand'] - min_d) / (max_d - min_d)

#Step 3: Compute dynamic price
lambda_coeff = 0.75
base_price = 10.0

df_model2['DemandPrice'] = (base_price * (1 + lambda_coeff * df_model2['NormalizedDemand']))
df_model2['DemandPrice'] = df_model2['DemandPrice'].clip(lower=5.0, upper=20.0)

#Step 4: Preview result
print(df_model2[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'QueueLength',
                 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType',
                 'RawDemand', 'NormalizedDemand', 'DemandPrice']].head(10))


  SystemCodeNumber           Timestamp  Occupancy  QueueLength  \
0      BHMBCCMKT01 2016-10-04 07:59:00         61            1   
1      BHMBCCMKT01 2016-10-04 08:25:00         64            1   
2      BHMBCCMKT01 2016-10-04 08:59:00         80            2   
3      BHMBCCMKT01 2016-10-04 09:32:00        107            2   
4      BHMBCCMKT01 2016-10-04 09:59:00        150            2   
5      BHMBCCMKT01 2016-10-04 10:26:00        177            3   
6      BHMBCCMKT01 2016-10-04 10:59:00        219            6   
7      BHMBCCMKT01 2016-10-04 11:25:00        247            5   
8      BHMBCCMKT01 2016-10-04 11:59:00        259            5   
9      BHMBCCMKT01 2016-10-04 12:29:00        266            8   

  TrafficConditionNearby  IsSpecialDay VehicleType  RawDemand  \
0                    low             0         car   0.958579   
1                    low             0         car   0.966378   
2                    low             0         car   1.807972   
3            

MODEL 3

In [13]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()

# Select one parking lot to visualize
sample_lot_id = df_model2['SystemCodeNumber'].unique()[0]
df_plot = df_model2[df_model2['SystemCodeNumber'] == sample_lot_id].sort_values('Timestamp')

# Create the figure
p = figure(x_axis_type='datetime', 
           title=f"Parking Price Over Time for Lot: {sample_lot_id}",
           height=400, width=800)

# Plot Baseline and Demand-Based Prices
p.line(df_plot['Timestamp'], df_plot['BaselinePrice'], line_width=2, color='blue', legend_label='Baseline Price')
p.line(df_plot['Timestamp'], df_plot['DemandPrice'], line_width=2, color='green', legend_label='Demand-Based Price')

# Add labels and legend
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"
p.legend.location = "top_left"
p.legend.click_policy = "hide"

# Show the plot
show(p)


Loading BokehJS ...